In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
folds = range(5)
mri_types_capital = ['flair','t1','t1ce','t2']
model_path = '/mnt/24CC5B14CC5ADF9A/Brain_Tumor_Classification/jupyter/Code/U-Net/Model/WB+WT/5Fold_scale_size100'

In [3]:
train_df = pd.read_csv('train_labels.csv',dtype = {'BraTS21ID':'str','MGMT_value':'int'})
index_name = train_df[(train_df['BraTS21ID'] == '00109') | (train_df['BraTS21ID'] == '00123') | (train_df['BraTS21ID'] == '00709')].index
train_df = train_df.drop(index_name).reset_index(drop=True)
train_df['MGMT_value'].value_counts()

1    306
0    276
Name: MGMT_value, dtype: int64

In [3]:
# test_df = pd.read_csv(f'{model_path}/flair/meta_classification_fold0.csv')
# valid_loss_min = test_df['valid loss'].values.min()
# valid_auc = test_df[test_df['valid loss']==valid_loss_min ]['valid AUC'].values
# valid_auc

In [4]:
# loss_mean = []
# loss_std = []
# auc_mean = []
# auc_std = []
# for mri_type in mri_types_capital:
#     loss_list = []
#     auc_list = []
#     for fold in folds:
#         result_df = pd.read_csv(f'{model_path}/{mri_type}/meta_classification_fold{fold}.csv')
#         valid_loss_min = result_df['valid loss'].values.min()
#         valid_auc = result_df[result_df['valid loss']==valid_loss_min ]['valid AUC'].values
#         loss_list.append(valid_loss_min)
#         auc_list.append(valid_auc)
#     loss_mean.append(np.mean(loss_list,dtype=np.float64))
#     loss_std.append(np.std(loss_list,dtype=np.float64))
#     auc_mean.append(np.mean(auc_list,dtype=np.float64))
#     auc_std.append(np.std(auc_list,dtype=np.float64))

In [5]:
# evaluation_metrics = {'MRI_Type':mri_types_capital,'Loss_Mean':loss_mean,'Loss_Std':loss_std,'AUC_Mean':auc_mean,'AUC_Std':auc_std}
# evaluation_metrics_df = pd.DataFrame(evaluation_metrics)
# evaluation_metrics_df.head()

In [4]:
import os
import re
import random
import pandas as pd
import numpy as np
import cv2

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils import data as torch_data
import time

import sklearn
from sklearn import model_selection as sk_model_selection
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from skimage.transform import resize

import nibabel as nib
import matplotlib.pyplot as plt

from unet_down import UNet

In [5]:
def set_seed(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        
def get_train_valid_split(label_path):
    train_df = pd.read_csv(label_path,dtype = {'BraTS21ID':'str','MGMT_value':'int'})
    index_name = train_df[(train_df['BraTS21ID'] == '00109') | (train_df['BraTS21ID'] == '00123') | (train_df['BraTS21ID'] == '00709')].index
    train_df = train_df.drop(index_name).reset_index(drop=True)

    X = train_df['BraTS21ID'].values
    y = train_df['MGMT_value'].values
    
    kfold =  StratifiedKFold(n_splits=5,shuffle = True,random_state = SEED)
    return X,y,list(kfold.split(X,y))


def load_raw_voxel(patient_id,mri_type):
    # Normalize voxel volume to 0~255
    voxels = nib.load(f'{RAW_DATA_PATH}/BraTS2021_{patient_id}/BraTS2021_{patient_id}_{mri_type}.nii.gz').get_fdata().astype('float')
    _min = voxels.min()
    _max = voxels.max()
    new_voxels = (voxels - _min) / (_max-_min) * 255.0
    return new_voxels

def load_mask(patient_id):
    return nib.load(f'{MASKS_PATH}/BraTS2021_{patient_id}.nii.gz').get_fdata().astype('float')


In [15]:
WB_PATH = '/mnt/24CC5B14CC5ADF9A/Brain_Tumor_Classification/Datasets/Data_WB_90_90_90'
WT_PATH = '/mnt/24CC5B14CC5ADF9A/Brain_Tumor_Classification/Datasets/Data_WT_90_90_90'
TC_PATH = '/mnt/24CC5B14CC5ADF9A/Brain_Tumor_Classification/Datasets/Data_TC_90_90_90'
ET_PATH = '/mnt/24CC5B14CC5ADF9A/Brain_Tumor_Classification/Datasets/Data_ET_90_90_90'


def non_0_voxel(voxel):
    length = voxel.shape[2]
    start_id = 0
    end_id = length-1

    # From begining to find start index
    for i in range(length):
        if np.max(voxel[:,:,i]) != 0:
            start_id = i
            break

    # From final to find end index
    for i in range(length-1,-1,-1):
        if np.max(voxel[:,:,i]) != 0:
            end_id = i
            break
    non_0_indexs = slice(start_id,end_id+1)
    
    return non_0_indexs 

# ---------------------- ET 90 90 90 ---------------------------
def construct_target_volume(scan_id,mri_type,scale_size=260):
    voxel_ET = nib.load(f'{ET_PATH}/BraTS2021_{scan_id}/BraTS2021_{scan_id}_{mri_type}.nii.gz').get_fdata().astype('float')
    return voxel_ET

"""
WB_PATH_96 = '/mnt/24CC5B14CC5ADF9A/Brain_Tumor_Classification/Datasets/Data_WB_96_96_62'

def construct_target_volume(scan_id,mri_type,scale_size=62):
    voxel_WB = nib.load(f'{WB_PATH_96}/BraTS2021_{scan_id}/BraTS2021_{scan_id}_{mri_type}.nii.gz').get_fdata().astype('float')
    return voxel_WB
"""
class Dataset(torch_data.Dataset):
    def __init__(self, ids, targets, mri_type, if_pred = False):
        self.ids = ids
        self.targets = targets
        self.mri_type = mri_type
        self.if_pred = if_pred
        
    def __len__(self):
        return len(self.ids)
    
    def __getitem__(self, index):
        scan_id = self.ids[index]
        data = construct_target_volume(scan_id,self.mri_type,scale_size=SCALE_SIZE)

        if self.if_pred:
            return {"X": torch.tensor(data).float().unsqueeze(0), "id":scan_id}
        else:
            y = torch.tensor(self.targets[index], dtype = torch.long)
            return {"X": torch.tensor(data).float().unsqueeze(0), "y": y}

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def predict(model_path,X_valid, y_valid,fold,mri_type):
    start = time.time()
    print(f"Predict: {mri_type} Fold:{fold}")

    # Load predict sample data
    data_retriever = Dataset(
        X_valid, 
        y_valid,
        mri_type,
    )
    
    data_loader = torch_data.DataLoader(
        data_retriever, 
        batch_size=2,
        shuffle=False,
        num_workers=4
    )
    model = UNet(in_channels=1,
                 out_channels=2,
                 n_blocks=4,
                 input_shape = INPUT_SIZE,
                 start_filters=32,
                 activation='relu',
                 normalization='batch',
                 conv_mode='same',
                 dim=3,
                 hidden_channels=2048)
    
    
    checkpoint = torch.load(f'{model_path}/{mri_type}/{mri_type}-fold{fold}-best.pth')
    model.load_state_dict(checkpoint["model_state_dict"])
    
    model.to(device)
    model.eval()
    
    y_all = []
    preds_all = []
    
    # Predict traninig samples
    for step, batch in enumerate(data_loader, 1):
        print(f"{step}/{len(data_loader)}", end="\r")
        with torch.no_grad():
            X = batch["X"].to(device)
            targets = batch["y"].to(device)
            outputs = model(X)
            
            # _, preds = torch.max(F.softmax(outputs,dim = 1), dim=1)
            preds = outputs.softmax(dim=1)[:,1]
            y_all.extend(batch["y"].tolist())
            preds_all.extend(preds.tolist())
    
    auc = roc_auc_score(y_all, preds_all)
    preds_labels = [1 if x > 0.5 else 0 for x in preds_all]
    f1 = f1_score(y_all,preds_labels)
    re = recall_score(y_all,preds_labels)
    pr = precision_score(y_all,preds_labels)
    del model
    print(f'Inference completed in {int(time.time()-start)}s')
    
    return auc,f1,re,pr

In [17]:
mri_types_capital = ['flair','t1','t1ce','t2']
model_path = '/mnt/24CC5B14CC5ADF9A/Brain_Tumor_Classification/jupyter/Code/U-Net/Model/ET_90_90_90'
RAW_DATA_PATH = '/mnt/24CC5B14CC5ADF9A/Brain_Tumor_Classification/Datasets/Data_Prep_Segmentation'
MASKS_PATH = '/mnt/24CC5B14CC5ADF9A/Brain_Tumor_Classification/Datasets/Data_Masks_Train'
LABEL_PATH = './train_labels.csv'
SCALE_SIZE = 90
SEED = 42

In [18]:
INPUT_SIZE = (90,90,90)
set_seed(SEED)

X,y,SPLIT = get_train_valid_split(LABEL_PATH)

In [19]:
def get_results(folds,mri_types):
    auc_mean = []
    auc_std = []
    f1_mean = []
    f1_std = []
    re_mean = []
    re_std = []
    pr_mean = []
    pr_std = []
    for mri_type in mri_types:
        auc_all = []
        f1_all = []
        re_all = []
        pr_all = []
        for fold in folds:
            train_idx,valid_idx = SPLIT[fold]
            X_valid,y_valid =X[valid_idx],y[valid_idx]
            auc,f1,re,pr = predict(model_path,X_valid, y_valid,fold,mri_type)
            auc_all.append(auc)
            f1_all.append(f1)
            re_all.append(re)
            pr_all.append(pr)
        auc_mean.append(np.mean(auc_all))
        auc_std.append(np.std(auc_all))
        f1_mean.append(np.mean(f1_all))
        f1_std.append(np.std(f1_all))
        re_mean.append(np.mean(re_all))
        re_std.append(np.std(re_all))
        pr_mean.append(np.mean(pr_all))
        pr_std.append(np.std(pr_all))
    
    evaluation_metrics = {'MRI_Type':mri_types,'AUC_Mean':auc_mean,'AUC_Std':auc_std,
                          'F1_Mean':f1_mean,'F1_Std':f1_std,
                         'Recall_Mean':re_mean,'Recall_Std':re_std,
                         'Precision_Mean':pr_mean,'Precision_Std':pr_std}
    evaluation_metrics_df = pd.DataFrame(evaluation_metrics)
    return evaluation_metrics_df

In [20]:
folds = range(5)
mri_types_capital = ['flair','t1','t1ce','t2']
df = get_results(folds,mri_types_capital)

Predict: flair Fold:0
Inference completed in 7s
Predict: flair Fold:1
Inference completed in 7s
Predict: flair Fold:2
Inference completed in 7s
Predict: flair Fold:3
Inference completed in 7s
Predict: flair Fold:4
Inference completed in 7s
Predict: t1 Fold:0
Inference completed in 7s
Predict: t1 Fold:1
Inference completed in 7s
Predict: t1 Fold:2
Inference completed in 7s
Predict: t1 Fold:3
Inference completed in 7s
Predict: t1 Fold:4
Inference completed in 7s
Predict: t1ce Fold:0
Inference completed in 7s
Predict: t1ce Fold:1
Inference completed in 7s
Predict: t1ce Fold:2
Inference completed in 7s
Predict: t1ce Fold:3
Inference completed in 7s
Predict: t1ce Fold:4
Inference completed in 7s
Predict: t2 Fold:0
Inference completed in 7s
Predict: t2 Fold:1
Inference completed in 8s
Predict: t2 Fold:2
Inference completed in 7s
Predict: t2 Fold:3
Inference completed in 7s
Predict: t2 Fold:4
Inference completed in 7s


In [21]:
df.head()

,MRI_Type,AUC_Mean,AUC_Std,F1_Mean,F1_Std,Recall_Mean,Recall_Std,Precision_Mean,Precision_Std
0,flair,0.589631,0.061707,0.640187,0.049420,0.738921,0.160776,0.588781,0.058559
1,t1,0.573924,0.064340,0.579431,0.082855,0.617715,0.159285,0.563353,0.054972
2,t1ce,0.576785,0.043543,0.572342,0.134325,0.643945,0.240302,0.551699,0.033949
3,t2,0.573502,0.059875,0.578157,0.075066,0.604654,0.128964,0.562887,0.036869


In [22]:
df.to_csv('./Results/ET_90_90_90.csv',index = False)